<a href="https://colab.research.google.com/github/jphermans/openprotocol/blob/main/OpenProtoCol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importeer de modules
import socket, base64, csv
import os, sys, time, argparse
from sys import platform
from subprocess import call

''' met deze parser gaan we argumenten aan het commando toevoegen
hier moeten ze dus het ipadres en poort toevoegen!'''
parser = argparse.ArgumentParser(description='Haal tooldata op via openprotocol')
parser.add_argument("host", help="pf ip adres", type=str)
parser.add_argument("port", help="pf port (4545)", type=int)
args = parser.parse_args()

encodedStr = "Q3JlYXRlZCBieSBKUEggMjAyMSAvIGplYW4tcGllcnJlLmhlcm1hbnNAYXRsYXNjb3Bjby5jb20="
# Standard Base64 Decoding
decodedBytes = base64.b64decode(encodedStr)
decodedStr = str(decodedBytes, "utf-8")

# maak bestand wanneer deze nog niet bestaat!
if not os.path.exists('tooldata.csv'):
  with open('tooldata.csv', 'w'): pass

def screen_clear():
  if platform == "linux" or platform == "linux2":
      #linux
    os.system('clear')
  elif platform == "darwin":
      #OSX
    os.system('clear')
  elif platform == "win32":
      #windows
    os.system('cls')
  else: #OS niet herkend!
    print("Operating System not regconized!")

'''haal adres en poort via commando
voor het aansturen via open protocol!'''
screen_clear() # maak scherm leeg!
hostpf = args.host # haal hostnaam of ip uit commando argument
pfpoort = args.port # haal de poort waarde uit commando argument

''' definieer variabelen [MIDS] voor gebruik. Het cijfer na de MIDx staat voor
het eigenlijke MID nummer dat we gaan gebruiken. ''' 
MID1 = '00200001001000000000'
MID40 = '00200040002000000000'
MID3 = '00200003001000000000'
MID270 = '270'

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
  sock.connect((hostpf, pfpoort))

  #stuur mid 1 om de connectie te testen
  sendMID1 = MID1 + chr(0)
  sock.sendall(sendMID1.encode())
  data1 = sock.recv(1024).decode()
  receivedMID = data1[4:8]
  
  #kijk received mid na of deze Ok is
  if receivedMID == '0002':
    print('[' + receivedMID + ']' + " - Ok telegram gekregen!")
    print()
    time.sleep(1.5)
    sendMID40 = MID40 + chr(0)
    sock.sendall(sendMID40.encode())
    data40 = sock.recv(1024).decode()
    sendMID3 = MID3 + chr(0)
    sock.sendall(sendMID3.encode())
    sock.close
    print("OS platform is: ", os.name)
    print('Received telegram ' + '[' + data40[4:8] + ']')
    print('*' * 50)
    print('              Tool serial: ', data40[22:36])
    print('        Total Tightenings: ', data40[38:48])
    print('           Last Cal. Date: ', data40[50:60])
    print('        Controller serial: ', data40[71:81])
    print('        Calibration value: ', data40[84:89])
    print('             Last service: ', data40[91:101])
    print('Tightenings since service: ', data40[113:122])
    print('      Controller firmware: ', data40[138:156])
    print('*' * 50)
    print(decodedStr)
    print("\n")
  else:
    print("Error in connection")
    time.sleep(2)
    sock.close
    sys.exit
